# Generate Artificial Faces with CelebA Progressive GAN Model


This exercise will let you use pre-trained models from tensorflow hub in order to generate images, the goal of the exercise will be to generate an image that is the most resembling to your own face!

The concepts here are a little challenging, this is on purpose, it's your last day of image processing with deep learning so we would like to show you some more complciated things, however most of the code here will be pre-written and you will just have to complete some cells to get it to work!

Two examples are provided:
* **Mapping** from latent space to images
* Given a target image, **using gradient descent to find** a latent vector that generates an image similar to the target image.

## Optional prerequisites

* Familiarity with [low level Tensorflow concepts](https://www.tensorflow.org/guide/eager).
* [Generative Adversarial Network](https://en.wikipedia.org/wiki/Generative_adversarial_network) on Wikipedia.
* Paper on Progressive GANs: [Progressive Growing of GANs for Improved Quality, Stability, and Variation](https://arxiv.org/abs/1710.10196).

### More models
[Here](https://tfhub.dev/s?module-type=image-generator) you can find all models currently hosted on [tfhub.dev](tfhub.dev) that can generate images.

## Setup

In [ ]:
# Install imageio for creating animations.  
!pip -q install imageio
!pip -q install scikit-image
!pip install git+https://github.com/tensorflow/docs

In [ ]:
from absl import logging

import imageio
import PIL.Image
import matplotlib.pyplot as plt
import numpy as np

import tensorflow as tf
#tf.random.set_seed(1)

import tensorflow_hub as hub
from tensorflow_docs.vis import embed
import time

try:
  from google.colab import files
except ImportError:
  pass

from IPython import display
from skimage import transform

## Random vectors

We will use a TF-Hub module [progan-128](https://tfhub.dev/google/progan-128/1) that contains a pre-trained Progressive GAN. Progressive GANs are GANs that were trained to progressively output higher and higher resolution images, in addition to the paper we mentionned earlier you may read <a href="https://machinelearningmastery.com/introduction-to-progressive-growing-generative-adversarial-networks/#:~:text=Progressive%20Growing%20GAN%20is%20an,output%20large%20high%2Dquality%20images.&text=Progressive%20growing%20GAN%20models%20are,resolution%20that%20are%20remarkably%20realistic."> this blog post </a> that explains very simply how this process works!

1. Use the documentation to load the model into an object called `progan`.

2. Use one of the attributes of the `progan` object to figure out what is the expected input shape for this model.

3. Create a random input vector to compute a random image using `progan`.

4. Use the following function to display your random image.

In [ ]:
# Simple way to display an image.
def display_image(image):
  image = tf.constant(image)
  image = tf.image.convert_image_dtype(image, tf.uint8)
  return PIL.Image.fromarray(image.numpy())



## Latent space interpolation

An interesting technique that can challenge the performance of your generative model is interpolation. The idea is the following: You generate two random vectors from the latent space (input vectors for the progan model), and you sort of draw an line between the two latent vectors and draw vectors from this line (as if you were trying to draw a dotted line between the two). Then you use the model to generate the images corresponding to all those latent vectors.

This will give us an idea of the richness of the output space of the progan model!

In this series of questions we will create a function `interpolate_hypersphere` that will generate all the vectors on the "dotted line" in between two vectors of the latent space, let's go!



1. We will use the `tf.norm` function, check out the documentation to see how this function works. It will be useful to normalize the vectors we are trying to interpolate. Compute the norm from the random vector we created before.

2. To implement the logic of interpolation, let's try it first on two 2-dimensional vectors. Create a numpy array with coordinates `(2,1)` and another with coordinates `(3,4)`, display them in a graph.

3. Try and create a hundred interpolation vectors between the two you have just created, and include them in a graph.

Congratulations you just did your first interpolation!

4. Now let's use the same principle to interpolate two vectors from the latent space of the progan model!

We'll use a specific type of interpolation here which is called hypersphere interpolation, the idea is that we'll have two vectors with different norms (the norm can be interpreted as the distance between a vector and the origin of the space i.e. the vector with all-zero coordinates) we normalize one of them so they have the same norm, then we interpolate the two vectors using only vectors with the same norm, which means all this vectors will be on the sphere centered on the origin and radius equals to the norm of the first vector! (Think of it as the trajectory of shortest distance between two cities on earth which is roughly a sphere!)

In [ ]:
# Interpolates between two vectors that are non-zero and don't both lie on a
# line going through origin. First normalizes v2 to have the same norm as v1. 
# Then interpolates between the two vectors on the hypersphere.
def interpolate_hypersphere(v1, v2, num_steps):
  # compute the norm for each vector
  v1_norm = 
  v2_norm = 
  # normalize v2 so it has the same norm as v1
  v2_normalized = 

  # We'll create an empty list where we will store all the interpolated vectors
  vectors = []
  # loop over the number of steps and create the collection of interpolated
  # vectors between v1 and nromalized v2
  for step in range(num_steps):
    # create the interpolated vector 
    interpolated = 
    # calculate the interpolated vector's norm
    interpolated_norm = 
    # normalize the interpolated vector so it has the same norm as v1
    interpolated_normalized = 
    # add that vector to the list
    vectors.append(interpolated_normalized)
  # Return a stacked version of all these vectors
  return tf.stack(vectors)


5. We introduce a utility function to render an animated gif showing the images produced from all the latent space vectors we use in the interpolation!

We'll then create an `interpolate_between_vectors` function that can produce all the images from two interpolated vectors.

In [ ]:
# Given a set of images, show an animation.
def animate(images):
  images = np.array(images)
  converted_images = np.clip(images * 255, 0, 255).astype(np.uint8)
  imageio.mimsave('./animation.gif', converted_images)
  return embed.embed_file('./animation.gif')

logging.set_verbosity(logging.ERROR)

In [ ]:
def interpolate_between_vectors():
  # create two random vectors from the latent space
  v1 = 
  v2 = 
    
  # Creates a tensor with 50 steps of interpolation between v1 and v2 using the 
  # interpolate_hypershpere function
  vectors = 

  # Use progan to generate images from the latent space.
  interpolated_images = 

  return interpolated_images

interpolated_images = interpolate_between_vectors()
animate(interpolated_images)

## Finding closest vector in latent space
Fix a target image. As an example use an image generated from the module or upload your own.
This next cell will let you pick the image you want to generate using the progan model:

In [ ]:
image_from_module_space = False  # @param { isTemplate:true, type:"boolean" }

def get_module_space_image():
  vector = tf.random.normal([1, latent_dim])
  images = progan(vector)['default'][0]
  return images

def upload_image():
  uploaded = files.upload()
  image = imageio.imread(uploaded[list(uploaded.keys())[0]])
  image = tf.cast(image/255, tf.float32)
  return transform.resize(image, [128, 128])

if image_from_module_space:
  target_image = get_module_space_image()
else:
  target_image = upload_image()

display_image(target_image)

After defining a loss function between the target image and the image generated by a latent space variable, we can use gradient descent to find variable values that minimize the loss.

1. Create an initial vector and display the image that progan is generating

2. Let's define a function that will find an image that ressembles the uploaded image as closely as possible.

For this we will use gradient descent (with custom training, you already knew this right?). The idea is to define a loss function that will measure how different the two images are from each other, and then use gradient descent to modify the latent vector so that the generated image approaches the target images as closely as possible. Let's do this!

In [ ]:
def find_closest_latent_vector(initial_vector, num_optimization_steps,
                               steps_per_image):
  # create two empty lists for the losses and images
  images = []
  losses = []

  # convert intital_vector into a variable tensor (because that's what we will
  # update through gradient descent)
  vector = 
  # set up an Adam optimizer
  optimizer = 
  # set up a loss function, you may use MeanAbsoluteError for now
  loss_fn = 

  # loop over num_optimization_steps
  for step in range(num_optimization_steps):
    #This will print a dot every 100 steps 
    if (step % 100)==0:
      print()
    print('.', end='')
    # Use gradient tape
    with 
      # Generate the image using the progan model
      image = 
      # Add the image to the list we'll visualize later
      if (step % steps_per_image) == 0:
        images.append(image.numpy())
      # compute the loss function between the generated and target image
      target_image_difference = 
      # The latent vectors were sampled from a normal distribution. We can get
      # more realistic images if we regularize the length of the latent vector to 
      # the average length of vector from this distribution.
      regularizer = tf.abs(tf.norm(vector) - np.sqrt(latent_dim))
      
      # add the regularization to the loss
      loss = 
      # append the loss to the losses list
      losses.append(loss.numpy())
    # compute the gradients thanks to the loss and according to the latent vector's
    # coordinates
    grads = 
    # use the optimizer to update the latent vector
    
    
  return images, losses

3. Use the `find_closest_latent_vector` function with 200 steps and 5 steps per image

In [ ]:
num_optimization_steps=200
steps_per_image=5
images, loss = find_closest_latent_vector(initial_vector, num_optimization_steps, steps_per_image)

4. Plot the evolution of the loss

5. Use the `animate` function in order to display the evolution of the generated image.

Compare the result to the target:

### Playing with the above example
If image is from the module space, the descent is quick and converges to a reasonable sample. Try out descending to an image that is **not from the module space**. The descent will only converge if the image is reasonably close to the space of training images.

How to make it descend faster and to a more realistic image? One can try:
* using different loss on the image difference, e.g. quadratic,
* using different regularizer on the latent vector,
* initializing from a random vector in multiple runs,
* etc.
